In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-07-06 15:18:15,952 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [2]:
kwargs = {
    "game": "iRacing",
    # "track": "oschersleben gp",
    # "track": "aragon gp",
    "track": "jerez gp",
    "car": "Ferrari 488 GT3 Evo 2020",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
# laps = list(laps)
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-07-06 15:18:16,049 DEBUG Influx: Connected to https://telemetry.b4mad.racing/
2023-07-06 15:18:16,053 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-06 15:18:16,053 INFO   track.id 61 car.id 9
2023-07-06 15:18:16,053 INFO   session 1688310090 lap.id 176859 number 3
2023-07-06 15:18:16,053 INFO   length 4368 time 102.7625 valid True
2023-07-06 15:18:16,054 INFO   start 2023-07-02 17:04:59.217731+00:00 end 2023-07-02 17:06:42.026664+00:00
2023-07-06 15:18:16,555 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-06 15:18:16,556 INFO   track.id 61 car.id 9
2023-07-06 15:18:16,556 INFO   session 1688223703 lap.id 173644 number 3
2023-07-06 15:18:16,556 INFO   length 4369 time 102.7718 valid True
2023-07-06 15:18:16,556 INFO   start 2023-07-01 17:05:13.753633+00:00 end 2023-07-01 17:06:56.538103+00:00
2023-07-06 15:18:17,033 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-06 15:18:17,034

In [3]:
# influx_data_frames = []
# for df in data_frames:
#     influx_data_frames.append(df.copy())

# data_frames = []
# for df in influx_data_frames:
#     data_frames.append(df.copy())

In [4]:
current_sectors = fast_lap_analyzer.current_fast_lap_sectors()
display(current_sectors)

[{'start': 0, 'end': 355},
 {'start': 356, 'end': 692},
 {'start': 693, 'end': 999},
 {'start': 1000, 'end': 1611},
 {'start': 1612, 'end': 2155},
 {'start': 2156, 'end': 2364},
 {'start': 2365, 'end': 2742},
 {'start': 2743, 'end': 3133},
 {'start': 3134, 'end': 3605},
 {'start': 3606, 'end': 4367}]

In [5]:
fig = lap_fig(data_frames[0], full_range=True, columns=["Throttle"])
fig.show()

In [11]:
# for i, df in enumerate(data_frames):
#     fig = lap_fig(df, full_range=True, columns=["Throttle"])
#     fig.show()
sector_start_end, df_max = fast_lap_analyzer.extract_sectors(data_frames)
display(sector_start_end)

# are the sectors similar to the current fast lap?
similarity = fast_lap_analyzer.similar_sectors(sector_start_end, current_sectors)
display(f"Sectors are similar: {similarity}")

fig = lap_fig(df_max, full_range=True, columns=["Throttle"])
# add a vertical line for each sector
for i, sector in enumerate(sector_start_end):
    fig_add_shape(fig, x0=sector["start"], x1=sector["start"], color="blue")
    fig_add_shape(fig, x0=sector["end"] - 1, x1=sector["end"] - 1, color="red")
fig.show()

2023-07-06 15:24:34,298 DEBUG Lap too short, skipping


[{'start': 0, 'end': 322.0, 'length': 322},
 {'start': 323.0, 'end': 747.0, 'length': 424},
 {'start': 748.0, 'end': 1028.0, 'length': 280},
 {'start': 1029.0, 'end': 1789.0, 'length': 760},
 {'start': 1790.0, 'end': 2134.0, 'length': 344},
 {'start': 2135.0, 'end': 2373.0, 'length': 238},
 {'start': 2374.0, 'end': 2752.0, 'length': 378},
 {'start': 2753.0, 'end': 3176.0, 'length': 423},
 {'start': 3177.0, 'end': 3724.0, 'length': 547},
 {'start': 3725.0, 'end': 4367.0, 'length': 642}]

2023-07-06 15:24:34,309 DEBUG start_diffs: [0, 33.0, 55.0, 29.0, 178.0, 21.0, 9.0, 10.0, 43.0, 119.0] med: 49.7


'Sectors are similar: False'

In [7]:
segments, used_laps = fast_lap_analyzer.extract_segments(sector_start_end, data_frames, laps_with_telemetry, df_max)
track_length = df_max["DistanceRoundTrack"].max()
for i in range(len(segments)):
    segment = segments[i]
    # display(segment.start, segment.end, segment.turn)
    # display(segment.telemetry)
    # df = telemetry_for_fig(segment)
    df = segment.telemetry
    fig = lap_fig(df, columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        # brake_features = features_for_fig(segment, track_length, brake_features)
        fig_add_features(fig, brake_features)
    throttle_features = segment.throttle_features()
    if throttle_features:
        # throttle_features = features_for_fig(segment, track_length, throttle_features)
        fig_add_features(fig, throttle_features, color="green")
    display(f"time: {segment.time} {segment.track_length}")
    fig.show()

2023-07-06 15:18:21,021 ERROR sector 8 is empty
2023-07-06 15:18:21,031 ERROR sector 8 is empty
2023-07-06 15:18:21,040 ERROR sector 8 is empty
2023-07-06 15:18:21,050 ERROR sector 8 is empty


'time: 8.236067 4368.0'

'time: 9.950000600000001 4368.0'

'time: 5.766666400000002 4368.0'

'time: 12.299999999999997 4368.0'

...

'time: 0.9333361999999994 4368.0'

'time: 7.799999200000002 4368.0'

'time: 9.966666099999998 4368.0'

'time: 12.983340000000013 4368.0'

'time: 17.899991 4368.0'